<a href="https://colab.research.google.com/github/justinballas/Articles/blob/main/BICFR_Snippets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is used as a repository of various blocks of code to be used in the Best In Class Fund Review (BICFR) workflow. This notebook is broken down into the following sections:

1. Importing the appropriate files

2. Data Cleaning

3. Carrier Subaccount Analysis and Rankings

4. Best in Class Fund Selection

5. Excel Reporting

In [2]:
!pip install xlsxwriter

     |████████████████████████████████| 153kB 7.5MB/s 


In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xlsxwriter
from glob import glob
%matplotlib inline

1. Importing the appropriate files

In [53]:
#Initiate a list of carriers. this abbreviation will be used to import each file based on the carrier name and will double as the alias for each carrier for use throughout the Python script. 
carriers_list = ['LCL', 'NWD', 'PL', 'PRT', 'Summit', 'MM', 'COIL']

#Using Glob to import files based on carrier name
files = [glob('{}*'.format(carrier)) for carrier in carriers_list]
#Glob returns a list, so when using list comprehentions it will return a list of lists, we must convert to a flat list.
files = [item for elem in files for item in elem]
#Reading in the CSVs, skipfooter removes the summary statistics from the Morningstar output
files = [pd.read_csv(file, skipfooter=13) for file in files]
#Converts to a dictionary to easily call DataFrames for each carrier. Dictionary structure is used for looping through different amounts carriers to increase reusability of scripts.
carrier_dfs = dict(zip(carriers_list, files))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


2. Data Cleaning

In [55]:
#removing "\n" anomaly from wrap text.
for carrier, df in carrier_dfs.items():
  df.rename(columns=lambda x: x.replace('\n',''), inplace=True)

In [56]:
#remove funds that are closed to new investment
for carrier in carriers_list:
  temp_df = carrier_dfs[carrier]
  temp_df.drop(temp_df[temp_df['Closed to New Inv']=='Yes'].index, inplace=True)

In [65]:
#Selecting columns used in BICFR
cols_list = ['Name',
 'Morningstar Category',
 'Advisor',
 'Subadvisor',
 'Total Ret 1 Yr (Mo-End) Base Currency',
 'Total Ret Annlzd 3 Yr (Mo-End) Base Currency',
 'Total Ret Annlzd 5 Yr (Mo-End) Base Currency',
 'Total Ret Annlzd 10 Yr (Mo-End) Base Currency',
 'Morningstar Rating Overall',
 'Morningstar Rating 3 Yr',
 'Morningstar Rating 5 Yr',
 'Morningstar Rating 10 Yr',
 'Total Ret % Rank Cat 1 Yr (Mo-End)',
 'Total Ret % Rank Cat 3 Yr (Mo-End)',
 'Total Ret % Rank Cat 5 Yr (Mo-End)',
 'Total Ret % Rank Cat 10 Yr (Mo-End)',
 'Manager Tenure (Average)',
 'Sharpe Ratio 3 Yr (Mo-End) Risk Currency',
 'Prospectus Net Expense Ratio']

for df in carrier_dfs.values():
  df = df.loc[:, cols_list]

In [71]:
for carrier, df in carrier_dfs.items():
  carrier_dfs[carrier] = df.loc[:,cols_list]

In [73]:
carrier_dfs['LCL'].columns

Index(['Name', 'Morningstar Category', 'Advisor', 'Subadvisor',
       'Total Ret 1 Yr (Mo-End) Base Currency',
       'Total Ret Annlzd 3 Yr (Mo-End) Base Currency',
       'Total Ret Annlzd 5 Yr (Mo-End) Base Currency',
       'Total Ret Annlzd 10 Yr (Mo-End) Base Currency',
       'Morningstar Rating Overall', 'Morningstar Rating 3 Yr',
       'Morningstar Rating 5 Yr', 'Morningstar Rating 10 Yr',
       'Total Ret % Rank Cat 1 Yr (Mo-End)',
       'Total Ret % Rank Cat 3 Yr (Mo-End)',
       'Total Ret % Rank Cat 5 Yr (Mo-End)',
       'Total Ret % Rank Cat 10 Yr (Mo-End)', 'Manager Tenure (Average)',
       'Sharpe Ratio 3 Yr (Mo-End) Risk Currency',
       'Prospectus Net Expense Ratio'],
      dtype='object')